In [ ]:
!pip3 install nltk==3.6.2

     |████████████████████████████████| 1.5 MB 4.1 MB/s 
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
import numpy as np
import itertools
import matplotlib
import nltk
import numpy as np
import itertools
import matplotlib
import nltk



def generate_pos_tags(test_sentence):
               
		from nltk.corpus import brown



		#Use universal tagset , currently not using
		sentence_tag = nltk.corpus.brown.tagged_sents(tagset="universal")
		modified_sentence_tag=[]
		for sent in sentence_tag:
			sent.insert(0,('^^','^^'))         # Sentence starts with '^^'
			sent.append(('$$','$$'))           # Sentence ends with '$$'
			modified_sentence_tag.append(sent)


		train_dataset = modified_sentence_tag

		modified_test_sentence = '^^ '+ test_sentence + ' $$'

		#needs to be created
		test_dataset = modified_test_sentence.split(' ')

		#Creation of a dictionary whose keys are tags and values contain words which have correspoding tag in the taining dataset
		#example:- 'TAG':{word1: count(word1,'TAG')} count(word1,'TAG') means how many times the word is tagged as 'TAG'
		train_word_tag = {}
		for sent in train_dataset:
			for (word,tag) in sent:
				word=word.lower()            # removing ambiguity from capital letters 
				try:
					try:
						train_word_tag[tag][word]+=1
					except:
						train_word_tag[tag][word]=1
				except:
					train_word_tag[tag]={word:1}


		#Calculation of emission probabilities using train_word_tag
		train_emission_prob={}
		for key in train_word_tag.keys():
			train_emission_prob[key]={}
			count = sum(train_word_tag[key].values())                           # count is total number of words tagged as a 'TAG'
			for key2 in train_word_tag[key].keys():
				train_emission_prob[key][key2]=train_word_tag[key][key2]/count    

		#Emission probability is #times a word occured as 'TAG' / total number of 'TAG' words
		#example: number of times 'Sandeep' occured as Noun / total number of nouns


		#Estimating the bigrams of tags to be used for calculation of transition probability 
		#Bigram Assumption is made, the current tag depends only on the previous tag
		bigram_tag_data = {}
		for sent in train_dataset:
			bi=list(nltk.bigrams(sent))
			for b1,b2 in bi:
				try:
					try:
						bigram_tag_data[b1[1]][b2[1]]+=1
					except:
						bigram_tag_data[b1[1]][b2[1]]=1
				except:
					bigram_tag_data[b1[1]]={b2[1]:1}
		#bigram_tag_data is storing the values for every tag.
		#Every key is a tag and value is tag followed for that key and corresponding counts.
		#example: how many times an adj is followed by a noun {Noun:{Adj:3}}, here its 3 times.

		#Calculation of the probabilities of tag bigrams for transition probability
		#We already made a bigram assumption   
		bigram_tag_prob={}
		for key in bigram_tag_data.keys():
			bigram_tag_prob[key]={}
			count=sum(bigram_tag_data[key].values())              # count is total number of times a 'TAG' has occured
			for key2 in bigram_tag_data[key].keys():
				bigram_tag_prob[key][key2]=bigram_tag_data[key][key2]/count
		#Tranmission probability is #times a TAG2 is preceded by TAG1 / total number of times TAG1 exists in dataset
		#example: number of times a noun occured before adjective / total number of times a noun occurred


		#Calculation the possible tags for each word in the entire dataset
		#Note: Here we have used the whole data(Train dataset + Test dataset)
		#Reason: Words present in Test data is not subset pf Train data.
		#The above thing can be neglected if not necessay, but it improves our accuracy of the model 
		tags_of_tokens = {}
		count=0
		for sent in train_dataset:
			for (word,tag) in sent:
				word=word.lower()
				try:
					if tag not in tags_of_tokens[word]:
						tags_of_tokens[word].append(tag)
				except:
					list_of_tags = []
					list_of_tags.append(tag)
					tags_of_tokens[word] = list_of_tags
		#Each word and its corresponding tags in the train dataset



		#Seperating the test data into test words
		test_words= [test_dataset]
		# print(test_words)

		
		predicted_tags = []                #Final list for prediction
		for i in range(len(test_words)):   # for each tokenized sentence in the test data (test_words is a list of lists)
			sent = test_words[i]
			#storing_values is a dictionary which stores the required values
			#ex: storing_values = {step_no.:{state1:[previous_best_state,value_of_the_state]}}                
			storing_values = {}              
			for q in range(len(sent)):
				step = sent[q]
				#for the starting word of the sentence
				if q == 1:                
					storing_values[q] = {}
					try:
						tags = tags_of_tokens[step]
					except:
						# print(step,test_tags_of_tokens[step])
						tags=['NOUN'] #tags_of_unseen_tokens
					for t in tags:
						#this is applied since we do not know whether the word in the test data is present in train data or not
						try:
							storing_values[q][t] = ['^^',bigram_tag_prob['^^'][t]*train_emission_prob[t][step]]
							#if word is not present in the train data but present in test data we assign a very low probability of 0.0001
						except:
							storing_values[q][t] = ['^^',0.0001]
		  
				  #if the word is not at the start of the sentence
				if q>1:
					storing_values[q] = {}
					previous_states = list(storing_values[q-1].keys())   # loading the previous states
					try:
						current_states  = tags_of_tokens[step]               # loading the current states
					except:
						current_states = ['NOUN']#tags_of_unseen_tokens
						#calculation of the best previous state for each current state and then storing
						#it in storing_values
					for t in current_states:                             
						temp = []
						for pt in previous_states:                         
							try:
							  temp.append(storing_values[q-1][pt][1]*bigram_tag_prob[pt][t]*train_emission_prob[t][step]) # If seen word
							except:
							  temp.append(storing_values[q-1][pt][1]*0.0001)
						max_temp_index = temp.index(max(temp))
						best_pt = previous_states[max_temp_index]
						storing_values[q][t]=[best_pt,max(temp)] #Store the best previous tag for each best candidate per tag and the meximum probability
		 
			#Backtracing to extract the best possible tags for the sentence
			# for each word looking the current word and the word and tag next to it in the sentence backtrack
			# to get the tag of current word
			pred_tags = [] #predicted tags by viterbi using backtracking
			total_steps_num = storing_values.keys()
			last_step_num = max(total_steps_num)     # Begin from the last word which will end the delimiter
			for bs in range(len(total_steps_num)):    
				step_num = last_step_num - bs          
				if step_num == last_step_num:
					pred_tags.append('$$')
					pred_tags.append(storing_values[step_num]['$$'][0]) 
				if step_num<last_step_num and step_num>0:
					pred_tags.append(storing_values[step_num][pred_tags[len(pred_tags)-1]][0]) #Looking into storing value fetch the best previous tag for the current word
			predicted_tags.append(list(reversed(pred_tags)))
		return predicted_tags

In [ ]:
import numpy as np
import nltk
from nltk.tree import Tree
from nltk.corpus import semcor
from gensim.models import KeyedVectors
from nltk.corpus import wordnet as wn

# Getting the brown corpus
print("Downloading files from NLTK please wait...")
nltk.download('all', quiet=True)
print("NLTK files downloaded!")


NLTK files downloaded!


In [ ]:
synCorpus=[]
current_index=0
for taggedSents in semcor.tagged_sents(tag='sem'): # for each sentence fetch the (phrase,synsetTree)
    synCorpus.append([])                           # create a list designating current sentence which will contain (phrase,synsetID)
    for phrase in taggedSents:
        if type(phrase)==list:                     # The phrase with no tag is a list in semcor [Note: CASE 1]
            pass
        else: 
            try: 
                # handle Case 2
                synCorpus[current_index].append((phrase.label().name(),phrase.label().synset().name())) 
            except:
                # handle Case 3, The tags with no synset id but a synset present
                pass
    current_index+=1 # Keeps track of the sentence index
no_sents=current_index #Saved total number of sentences

In [ ]:
print("Loading word2vec")
!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
try:
	model_w2v = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz',binary=True)
except:
	print("Download pretrained word2vec from https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz and save in the same directory of the file\nExiting...")
	exit()


Loading word2vec
--2021-11-03 18:40:13--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.227.152
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.227.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  16.4MB/s    in 1m 53s  

2021-11-03 18:42:06 (14.0 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



TEST FUNCTION

In [ ]:
def test(sentence):
  one_sentence=[]
  text = nltk.word_tokenize(sentence)
  sentence_tags=nltk.pos_tag(text)
  sentence_list=[]
  for word in text:
    sentence_list.append((word,"dummy"))
  one_sentence.append(sentence_list)

  import numpy as np
  np.seterr(divide='ignore', invalid='ignore')
  synCorpus_words=[]
  line=0


  for sent in one_sentence:
    synCorpus_words.append([])
    for (word,tag) in sent:
        synCorpus_words[line].append(word)
    line+=1

  # Find context bag for each sentence, the ambiguous word is ignored when comparing with the ambigous word's sense
  length_of_wordvec=len(model_w2v["the"])
  context_bag_vector = []
  for sent in synCorpus_words:
      context_count=0
      context_list=np.zeros((length_of_wordvec,))
      #for every sentence get the context bag
      single_sentence='_'.join(word for word in sent)
      single_sentence=single_sentence.replace('-','_')
      splits=single_sentence.split('_')
      for word in splits:
          try:
              context_list+=model_w2v[word]
              context_count+=1
          except:
              pass
      if context_count!=0:
          context_bag_vector.append(context_list/context_count)
      else:
          context_bag_vector.append(context_list)


  # Find similarity with sense bag for each sense
  overlapCorpus=[]
  count=0
  test=0
  test1=0
  for sent in synCorpus_words:
      overlapCorpus.append([])
      sentence_pos_tags=generate_pos_tags(sentence)[0][1:-1]
      curr_word_index=-1
      for word in sent:
          curr_word_index+=1
          synsets_word=wn.synsets(word)
          if len(synsets_word) ==0:
            overlapCorpus[count].append((word,'NOT IN WORDNET'))
            continue
          else:
            if sentence_pos_tags[curr_word_index]=='NOUN':
              if len(wn.synsets(word,pos=wn.NOUN))!=0:
                synsets_word=wn.synsets(word,pos=wn.NOUN)
            elif sentence_pos_tags[curr_word_index]=='ADJ':
              if len(wn.synsets(word,pos=wn.ADJ))!=0:
                synsets_word=wn.synsets(word,pos=wn.ADJ)
            elif sentence_pos_tags[curr_word_index]=='ADV':
              if len(wn.synsets(word,pos=wn.ADV))!=0:
                synsets_word=wn.synsets(word,pos=wn.ADV)
            elif sentence_pos_tags[curr_word_index]=='VERB':
              if len(wn.synsets(word,pos=wn.VERB))!=0:
                synsets_word=wn.synsets(word,pos=wn.VERB)
            else:
              pass
          curr_sim=-1
          try:
            curr_synset=synsets_word[0]
          except:
            pass
          for synset_word in synsets_word: # for each sense of a word
              amb_list=np.zeros((length_of_wordvec,))
              amb_count=0
              synset_gloss=synset_word.definition() # get the word's gloss
              all_eg=synset_word.examples()
              synset_all_eg=""
              for i in range(len(all_eg)):
                if all_eg[i]==' ':
                  break
                synset_all_eg+=" "+all_eg[i]
              synset_gloss+=" "+synset_all_eg
              splits=synset_gloss.split()
              for word1 in splits:
                  try:
                      if(word1!=word):
                          amb_list+=model_w2v[word1]
                          amb_count+=1
                  except:
                      pass
              if amb_count!=0: # for context bag wrt an ambiguous word
                  amb_vec=(amb_list/amb_count)
              else:
                  amb_vec=amb_list
              cos_similarity=model_w2v.cosine_similarities(context_bag_vector[count].T,[amb_vec.T])[0] # Check cosine similary
              if curr_sim<cos_similarity: # and update the sense to the most similar sense
                  curr_sim=cos_similarity
                  curr_synset=synset_word
          overlapCorpus[count].append((word,curr_synset.definition()))
      count+=1

  return overlapCorpus[0]

In [ ]:
test("I went to the bank to withdraw some money.")

[['^^', 'I', 'went', 'to', 'the', 'bank', 'to', 'withdraw', 'some', 'money.', '$$']]


[('I', 'the smallest whole number or a numeral representing this number'),
 ('went', "have a turn; make one's move in a game"),
 ('to', 'NOT IN WORDNET'),
 ('the', 'NOT IN WORDNET'),
 ('bank',
  'a financial institution that accepts deposits and channels the money into lending activities'),
 ('to', 'NOT IN WORDNET'),
 ('withdraw', 'pull back or move away or backward'),
 ('some', 'relatively much but unspecified in amount or extent'),
 ('money', 'the official currency issued by a government or national bank'),
 ('.', 'NOT IN WORDNET')]